# Lab 4: Probing the capabilities of LLMs

Unlike previous assignments in this course, our primary goal in this lab is not to use NLP tools and techniques model language *per se*, but rather to investigate properties of language models themselves, since large language models (LLMs) are "black boxes" whose inner workings cannot be directly observed.

In particular, you will utilize and interpret the outputs of language models to **probe** features of those models--and in particular, how closely (or not) they resemble human language use/knowledge. We will design and implement probes for masked language modeling in BERT, in order to build on our knowledge from Lab 3, but these techniques are very generally applicable to models of all sorts, including generative models like GPT-3.




Masked Language Modeling is essentially a game of "fill in the blank". The model is given an input text of which a portion is "masked", and trained to predict what the masked element is given the surrounding context (both before and after the mask). Your job is to use these predictions to reason about how the model itself works.

# Rules
* The assignment should submitted to **Blackboard** as `.ipynb`. Only **one submission per group**.

* The **filename** should be the group number, e.g., `01.ipynb` or `31.ipynb`.

* The questions marked **Extra** or **Optional** are an additional challenge for those interested in going the extra mile. There are no points for them.

**Rules for implementation**

* You should **write your answers in this iPython Notebook**. (See http://ipython.org/notebook.html for reference material.) If you have problems, please contact your teaching assistant.

* Use only **one cell for markdown** answers!    

    * You do **not need to submit any code** for this lab, but you are free to leave any code you might run in your submission, so long as it does not interfere with readability of your written responses.
    * For text-based questions, put your solution in the `█████ YOUR ANSWER HERE █████` cell and keep the header.

* Don't change or delete any initially provided cells, either text or code, unless explicitly instructed to do so.
* Don't change the names of provided functions and variables or arguments of the functions.
* Leave the output of your code in the output cells.
* Test your code and **make sure we can run your notebook** in the colab environment.
* Don't forget to fill in the contribution information.

<font color="red">You following these rules helps us to grade the submissions relatively efficiently. If these rules are violated, a submission will be subject to penalty points.</font>  

# <font color="red">Contributions</font>

~~Delete this text and write instead of it:~~
* ~~a list of group members names (NOT student IDs)~~
* ~~who contributed to which exercises (you don't need to be very detailed)~~

# Setup

BERT, which you are already familiar with, is pre-trained on a masked language modeling task. We will use this model to make predictions about what "fills in the blank" in a masked language task. As before, we will need the transformers package to use BERT.

In [1]:
# !pip install transformers
import transformers
print(transformers.__version__) # 4.41.2

4.41.2


Then, we need to instantiate the tokenizer and the masked learning model.

In [2]:
import torch
from transformers import BertTokenizer, BertModel, BertForMaskedLM, RobertaTokenizer, RobertaModel, RobertaForMaskedLM

seed = 5
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertForMaskedLM.from_pretrained('bert-base-uncased')

#bert_model.eval()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


# Dealing with masked sentences

Next, we want to define some methods to allow us to see the probability of particular candidate tokens to "fill in the blank" in some text. We will use the token [MASK] to denote the blank.

The class ```MaskedSentence``` takes a sentence with a [MASK] token and uses softmax to turn the weights of all possible predicted values for the mask into probabilities. The class has two additional methods:

* The function ```get_masked_token_probability``` takes a string  ```token``` and prints the likelihood that BERT assigns to [MASK] being replaced in the text with ```token```.
* The function ```predict_masked_sentence``` prints the top *k* (5 by default) predictions for [MASK] with their probabilities.

In [3]:
# adapted from code by Yuchen Liu

class MaskedSentence:

  """
  A tokenized sentence with a masked word
  Note: [MASK] is the default mask token for BERT, other MLMs may have different defaults
  """

  def __init__(self, text, model=bert_model, tokenizer=bert_tokenizer, mask_token="[MASK]"):

    # Tokenize text and obtain predictions for mask

    self.tokenizer = tokenizer
    self.mask = mask_token

    text = "[CLS] %s [SEP]"%text
    tokenized_text = self.tokenizer.tokenize(text)
    masked_index = tokenized_text.index(mask_token)
    indexed_tokens = self.tokenizer.convert_tokens_to_ids(tokenized_text)
    tokens_tensor = torch.tensor([indexed_tokens])

    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]

    # Turn predictions into a probability distribution using softmax

    self.probs = torch.nn.functional.softmax(predictions[0, masked_index], dim=-1)

  def get_masked_token_probability(self, token):

    # prints probability of mask being replaced by token

    token_id = self.tokenizer.convert_tokens_to_ids(token)
    token_prob = self.probs[token_id]

    print(f"{self.mask}: {token},  | probability:, {float(token_prob)}, \n")

  def predict_masked_sent(self, top_k=5):

    # prints k most probable replacements for token

    top_k_weights, top_k_indices = torch.topk(self.probs, top_k, sorted=True)

    for i, pred_idx in enumerate(top_k_indices):
        predicted_token = self.tokenizer.convert_ids_to_tokens([pred_idx])[0]
        token_weight = top_k_weights[i]
        print(f"{self.mask}: {predicted_token}, | probability:, {float(token_weight)}")


Now, let's test these methods on a string with a mask.

In [4]:
test_sentence = MaskedSentence("All the world’s a [MASK], and all the men and women merely players.")
test_sentence.get_masked_token_probability("stage")
test_sentence.predict_masked_sent(top_k=5)

[MASK]: stage,  | probability:, 0.0009108720696531236, 

[MASK]: game, | probability:, 0.22936248779296875
[MASK]: team, | probability:, 0.2215639054775238
[MASK]: player, | probability:, 0.1818789690732956
[MASK]: champion, | probability:, 0.02110450714826584
[MASK]: winner, | probability:, 0.012764733284711838


We see that the model assigns a probability of about 0.0009 to *stage*, and the highest-probability token is *game* (0.229).

# Testing linguistic knowledge in MLMs

A major outstanding question in the study of large language models in general is how well (or poorly) the models are able to replicate aspects of humans' implicit linguistic knowledge and linguistic reasoning. One way to test this is to see how the model predicts a target word.

For example, consider the ways we could fill in the blank in the sentence *If cats were herbivores, they would probably eat _________.* This sentence is an example of what linguists call a **counterfactual conditional**: a description of what would happen if some hypothetical (but contrary to reality) condition were met. A speaker of English could recognize that this sentence is describing a hypothetical situation in which cats eat only plants, so the most logical continuation would be a word that describes edible plants, like *vegetables* or *carrots*.

Let's see what BERT predicts as the likeliest possible predictions for the mask:

In [5]:
cats_sent = MaskedSentence("time flies like an arrow and fruit flies like a [MASK].")
cats_sent.predict_masked_sent(top_k=5)


#"time flies like an arrow and fruit flies like a [MASK]."
#"If cats were herbivores, they would probably eat [MASK]."
#"herbivores eat plants. cats are not herbivores. However, if cats were herbivores, they would probably eat [MASK]."

[MASK]: bird, | probability:, 0.07395214587450027
[MASK]: bee, | probability:, 0.055763211101293564
[MASK]: fly, | probability:, 0.044657859951257706
[MASK]: fruit, | probability:, 0.03475882112979889
[MASK]: flower, | probability:, 0.03056127019226551


We see that BERT's top 5 predictions for the mask are *them, humans, it*, *animals*, and *meat*. On one hand, all of these predictions result in grammatical (syntactically well-formed) sentences, but they are either not very contentful (*them*, *it*) or nonsensical (*humans*, *meat*, *animals*)---none human-like. By contrast, the probabilities of *vegetables* and *carrots* are both relatively low:

In [ ]:
cats_sent.get_masked_token_probability("vegetables")
cats_sent.get_masked_token_probability("carrots")

[MASK]: vegetables,  | probability:, 0.0011069991160184145, 

[MASK]: carrots,  | probability:, 2.656224978636601e-06, 



# Ex 1 [1pt] Evaluating counterfactual conditionals in BERT

Provide a possible explanation, in 150-200 words, as to why BERT gives such non-human like results for this counterfactual conditional sentence. Your explanation should address the following questions: Do you think this is an arbitrary feature of this one sentence? Or does it reveal something more general about BERT? How could you go about testing whether your explanation is correct, using the class defined above?


<font color="red">█████ YOUR ANSWER HERE █████</font>

According to a recent paper (arXiv:2305.16572), there are a few factors which contribute to this behavior. The paper tests the capacity of pre-trained models including GPT3 and BERT on predicting the consequences of counterfactual conditionals. They find BERT to perform the worst in general and when compared to GPT3. The authors show that models which have a stronger grasp of relevant world knowledge are better equipped to manage counterfactuals. They also find combining relevant world knowledge with simple lexical triggers produce better predictions. The authors further show that, compared to GPT3, a model like BERT largely relies on simple lexical triggers in its predictions of counterfactuals. Based on this paper, we can attribute BERT's poor performance on counterfactuals to a lower grasp of world knowledge and stronger reliance on only structural cues. We can test this by feeding the relevant knowledge to BERT and then prompting it to predict the most likely words. For example, if we modify the sentence above to "Herbivores eat plants. cats are not herbivores. However, if cats were herbivores, they would probably eat [MASK].", the likelihood of the masked word being "plants" increases to 61%. When we review the BERT paper (Devlin et al. 2019) we can see that fine-tuning the pre-trained model on the prediction task given in these examples of counterfactuals is very likely to improve the performance of the model.

# Ex2 [4pt] Design your own BERT probe experiment

We can reason about the capabilities of an LLM simply by choosing carefully designed inputs and evaluating the model's corresponding outputs. If we have test many inputs with some shared property, such as a particular syntactic structure, we can start to generalize about BERT's behaviour with text that has that property. For example, we can investigate the question of whether or not BERT predicts continuations of counterfactual conditionals which are consistent with the hypothetical scenario presented in the *if*-clause of the conditional by evaluating what happens when we give it many such conditionals as inputs.

Your primary task for this final lab is to design a small experiment that tests, using the same kinds of techniques as above, the capabilities of BERT in a particular domain of your choice. To give you some ideas, here are a few suggestions of possible general domains that could be worth investigating, although the actual question you investigate should be small enough that it can be tested with a relatively modest selection of sentences. You are also free to come up with your own idea:

* The interpretation of pronouns (can BERT recognize which individual a pronoun like *it* is referring to when there are multiple possible options?)
* Does BERT fall for so-called "semantic illusions", in which it fails to recognize an inaccuracy in text, such as answering the question "How many of each animal did Moses take on the ark?" with "2"? (The Biblical story is about Noah, not Moses.)
* Bias: Does BERT make predictions which are more consistent with gender, racial, or other stereotypes?
* World knowledge: Does BERT make predictions which correspond with the way the world actually is?

Your description of your experiment should have the following parts and be approximately equivalent in length to 1 typed page (roughly 500 words, in addition to your test sentences):



*   **Research Question**: A clear formulation of the question you intend to investigate. It should be small and precise enough that it can reasonably be investigated using the functions defined above.
*   **Hypothesis**: The answer to the research question that you predict to be true, and *why you have that specific expectation*.
*   **At least 10 test sentences**, with a description of which of their properties are relevant. Be very clear about what, specifically, you are testing, and how the results will bear on your hypothesis.
*   **Test** your sentences and see what outputs you get. Do these provide evidence for or against your hypothesis? Why do you think you got the results you did?  

*   **Discuss** whether, given your own linguistic intuitions, the behaviour of BERT approximates that of a human language user with respect to your research question. If it is not human-like, how could the model be improved (in terms of training data, architecture, etc.) to achieve better results?
* **OPTIONAL**: Try investigating your research question in some other models (see https://huggingface.co/models for some options). You will likely need to adapt your probes for other kinds of models--for instance, a probe you test in a dialogue-based interface like ChatGPT will be different than those you designed for BERT. Some models, like [RoBERTa](https://huggingface.co/FacebookAI/roberta-base), are built upon the BERT architecture, so they are compatible with the code above (see example below--note that RoBERTa tokenizes slightly differently from BERT).

In [ ]:
roberta_tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
roberta_model = RobertaForMaskedLM.from_pretrained('roberta-base')

#Note 1: RoBERTa's default mask token is <mask>
#Note 2: Most tokens in RoBERTa begin with the unusual character Ġ.
#This is an artefact of the tokenization process, which includes the space preceding words.
#To get the probability of a token "word",
#we need to give get_masked_token_probability "Ġword"

roberta_test_sentence = MaskedSentence("All the world’s a <mask>, and all the men and women merely players.", model=roberta_model, tokenizer=roberta_tokenizer, mask_token='<mask>')
roberta_test_sentence.get_masked_token_probability("Ġstage")
roberta_test_sentence.predict_masked_sent(top_k=5)

<font color="red">█████ YOUR ANSWER HERE █████</font>

**Research Question:**
To what extent does BERT demonstrate proficiency in understanding and solving mathematical computations?
Our motivation is that BERT being a text-based, unimodal, model may fail to recognize mathematical constructs in its pre-training stage. We faced a similar issue with ChatGPT. We may, a priori, theorize that a transformer model trained on text is likely to fail at recognizing mathematical constructs because they differ markedly from linguistic constructs. Thus, if BERT treats numbers as contextualized word vectors, it’s likely to fail at mathematical tasks unless it is fine-tuned at such tasks.  

**Hypothesis:**
Since BERT is trained to recognize linguistic constructs and treats all its inputs as contextualized word embeddings, it’s likely to fail at mathematical tasks unless it’s fine-tuned to handle such tasks.
Given that the exercise asks for at least 10 test sentences, we will hypothesize that BERT will fail at more than 2 mathematical tasks out of 10. Thus, our confidence level is 80%.

**At least 10 test sentences**

The sentences below test different aspects of mathematics, from basic math to sequences and slightly more complicated mathematical problems. To ensure that BERT's results pertain to knowledge of mathematics and not simply linguistic knowledge, we vary the test sentences to capture the specific mathematical results as shown in examples 6, 7, 9, and 10 below.

1.	Simple arithmetic in a story format: “If Sarah has three candles and she buys four more candles, she will own [MASK] candles.”
2.	Simple arithmetic in a numerical format: “Sarah bought 2 grams of apples and later bought 3 more grams of apples. In total, Sarah bought [MASK] grams of apples."
3.	Direct numerical example: "the result of adding two to two is [MASK]."
4.	Testing a numerical sequence with no gaps: "If three follows after two, then four follows after [MASK]."
5.	Testing a numerical sequence with a gap: "If three follows after two, then ten follows after [MASK]."
6.	Recognizing mathematics with no ambiguities: "If three is a number, then twenty is a [MASK]."
7.	Recognizing mathematics with ambiguities: "If three is a number, then car is a [MASK]."
8.	Testing a slightly more complicated mathematical problem: "Robert earned 20 euros. He spent 5 euros on apples and was given 10 euros. Robert ended up with [MASK] euros."
9.	General mathematical facts with a forward sequence: "a circle is a 2 dimensional object while a sphere is a [MASK] dimensional object."
10.	General mathematical facts with a backward sequence: "a circle is a 2 dimensional object while a line is a [MASK] dimensional object."

**Test**
We tested BERT’s ability to recognize mathematical constructs and mathematical problems with the above sentences. We computed the probability distribution of the most likely masked words for each test sentence. BERT failed at all 10 sentences. We found that although it may appear that BERT is able to correctly predict the most likely masked word in the ninth sentence, it only does so because of the linguistic structure of the sentence and not by recognizing any mathematical patterns. This observation is validated by realizing that BERT produces almost the probability distribution for the tenth sentence.  

**Discuss**
The results confirm our hypothesis that BERT, being a language model, is unable to recognize mathematical constructs or even mathematical facts. We believe, however, that it’s possible for fine-tuning to make BERT better able at handling mathematical tasks. Pre-trained models such as BERT are particularly good at recognizing linguistics patterns given their training on a very large corpus. However, they often need additional fine-tuning to handle specific tasks.



# Acknowledgments

Concept and lab designed by Tom Roberts. BERT MLM script was heavily based on work by Yuchen Lin. Counterfactual example adapted from [Li, Yu, and Ettinger (2023)](https://aclanthology.org/2023.acl-short.70.pdf).